IMPORT LIBRERIE

In [51]:
from IPython.display import Image
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import model_from_json
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
from random import randint
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline
from skimage.io import imread

LOAD MODEL

In [52]:
#load model dalla cartella Models
json_file = open('..\\Models\\model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("..\\Models\\model.h5")
print("Loaded model from disk")

Loaded model from disk


INFERENCE

In [53]:
#costituisco la directory Inference e seleziono un'immagine dalla cartella nok a scopo di esempio

os.chdir('..\\Data\\Prepared\\ok')
if os.path.isdir('..\\..\\Inference') is False:
    os.makedirs('..\\..\\Inference')
    
    for i in random.sample(glob.glob('ok*'), 0):     #sostituire con 0 lo 1 in caso si voglia un immagine nok
        shutil.move(i, '..\\..\\Inference')
os.chdir('..\\nok')
for i in random.sample(glob.glob('nok*'), 1):        #sostituire con 1 lo 0 in caso si voglia un immagine nok
    shutil.move(i, '..\\..\\Inference')  
os.chdir('../../')

In [54]:
#rename file presenti in Inference

def main():
   
    folder = "..\\Data\\Inference"
    for count, filename in enumerate(os.listdir(folder)):
        dst = f"pred{str(count)}.bmp"
        src =f"{folder}/{filename}"  
        dst =f"{folder}/{dst}"
        os.rename(src, dst)
        
if __name__ == '__main__':
    main()

In [55]:
#deploy di un immagine ok
pred_filenames = os.listdir("..\\Data\\Inference")
pred_df = pd.DataFrame({'filename': pred_filenames})
nb_samples = pred_df.shape[0]
batch_size = 10
pred_gen = ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_v3.preprocess_input)
pred_generator = pred_gen.flow_from_dataframe(pred_df, "..\\Data\\Inference", 
                                              x_col='filename',
                                              y_col=None,
                                              class_mode=None,
                                              batch_size=batch_size,
                                              target_size=(299, 299),
                                              shuffle=False)
X_pred = pred_generator
pred = loaded_model.predict(X_pred, verbose=0)
rounded_pred = np.argmax(np.round(pred), axis=-1) 
if rounded_pred[0] == 0:
    print("\033[1m The algorithm predicts, with a probability of", np.round((pred[0,0])*100,2) ,"% that this is an ok image")
else:
    print("\033[1m The algorithm predicts, with a probability of", np.round((pred[0,1])*100,2) ,"% that this is a nok image")

Found 1 validated image filenames.


c:\Users\lopi1\anaconda3\lib\site-packages\keras\engine\training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


 The algorithm predicts, with a probability of 100.0 % that this is a nok image


In [50]:
#rimuovo la directory per permettere al processo di essere ripetuto
shutil.rmtree('..\\Data\\Inference')